# Ginny & Evan: single AND gate
This notebook walks through the classical Yao garbled AND example from Section 1.1 of Sophia Yakoubov’s “A Gentle Introduction to Yao’s Garbled Circuits” ([PDF](https://web.mit.edu/sonka89/www/papers/2017ygc.pdf)), showing how Ginny garbles, how labels are delivered (toy OT), how Evan evaluates once, and how the output label is returned and decoded.

In [11]:
:dep rand = "0.8"
:dep sha2 = "0.10"
:dep base64 = "0.21"

use rand::{rngs::OsRng, seq::SliceRandom, RngCore};
use sha2::{Digest, Sha256};
use base64::{engine::general_purpose::STANDARD_NO_PAD as B64, Engine};

#[derive(Clone, Debug)]
struct WireLabels {
    zero: [u8; 16],
    one: [u8; 16],
}

fn rand_label() -> [u8; 16] {
    let mut bytes = [0u8; 16];
    OsRng.fill_bytes(&mut bytes);
    bytes
}

fn h(k1: &[u8], k2: &[u8]) -> [u8; 32] {
    let mut hasher = Sha256::new();
    hasher.update(k1);
    hasher.update(k2);
    hasher.finalize().into()
}

fn encrypt(key: &[u8; 32], msg: &[u8; 16]) -> [u8; 16] {
    let mut out = [0u8; 16];
    for i in 0..16 {
        out[i] = msg[i] ^ key[i];
    }
    out
}

type Cipher = [u8; 16];


Garbled AND gate:

In [12]:
struct GarbledAnd {
    table: Vec<Cipher>,
    out_labels: WireLabels,
    input_a: WireLabels,
    input_b: WireLabels,
}

// Construct one garbled AND gate from fresh labels, shuffling the table.
fn garble_and() -> GarbledAnd {
    let input_a = WireLabels {
        zero: rand_label(),
        one: rand_label(),
    };
    let input_b = WireLabels {
        zero: rand_label(),
        one: rand_label(),
    };
    let out_labels = WireLabels {
        zero: rand_label(),
        one: rand_label(),
    };

    let mut table = Vec::new();
    for a in 0..=1 {
        for b in 0..=1 {
            let key = h(
                if a == 0 { &input_a.zero } else { &input_a.one },
                if b == 0 { &input_b.zero } else { &input_b.one },
            );
            let out = if (a & b) == 1 {
                &out_labels.one
            } else {
                &out_labels.zero
            };
            table.push(encrypt(&key, out));
        }
    }

    let mut rng = rand::thread_rng();
    table.shuffle(&mut rng);

    GarbledAnd {
        table,
        out_labels,
        input_a,
        input_b,
    }
}


In [13]:
// Generate one garbled AND gate instance.
let garbled = garble_and();
println!("Garbled AND generated; ready to evaluate in the next cell.");


Garbled AND generated; ready to evaluate in the next cell.


Inspect the garbled table before any party chooses inputs.


In [14]:
{
    println!("Ciphertexts (shuffled, base64) before inputs are chosen:");
    for (i, ct) in garbled.table.iter().enumerate() {
        println!("  row {i}: `{}`", B64.encode(ct));
    }

    println!("\nInput labels (base64) known to both parties (via OT/pre-share):");
    println!("  Ginny g=0: `{}`", B64.encode(garbled.input_a.zero));
    println!("  Ginny g=1: `{}`", B64.encode(garbled.input_a.one));
    println!("  Evan  e=0: `{}`", B64.encode(garbled.input_b.zero));
    println!("  Evan  e=1: `{}`", B64.encode(garbled.input_b.one));
}


Ciphertexts (shuffled, base64) before inputs are chosen:
  row 0: `+AoMmlR4zj47DS0Esx430Q`
  row 1: `5mJWvgjCjOgxxNoRUWmSVA`
  row 2: `7P21WtK+qY0WzR2cc5iCCw`
  row 3: `dDu3BCOBbXFXJb5QTRZS0g`

Input labels (base64) known to both parties (via OT/pre-share):
  Ginny g=0: `3nhaEbK83PjiU1RLpOzM0g`
  Ginny g=1: `FOi8AbzcY5FkEvJVyoTepQ`
  Evan  e=0: `bL1zclBA2RXXL3IXR4uSaw`
  Evan  e=1: `5w14t8H/NRrFKWXidM7HuQ`


()

Ginny now chooses a label; we assume a secure, authenticated channel when she sends it to Evan.

In [15]:
let ginny_pref: u8 = if rand::random::<bool>() { 1 } else { 0 }; // Ginny decides now
let ginny_label: [u8; 16] = if ginny_pref == 0 {
    garbled.input_a.zero
} else {
    garbled.input_a.one
};

// Secure, authenticated channel is assumed; send Ginny's chosen label directly.
println!("Ginny preference g={ginny_pref}; sending label over the secure channel:");
println!("  label (base64): `{}`", B64.encode(ginny_label));
println!("Evan receives the correct Ginny label over the secure channel.");


Ginny preference g=1; sending label over the secure channel:
  label (base64): `FOi8AbzcY5FkEvJVyoTepQ`
Evan receives the correct Ginny label over the secure channel.


Oblivious transfer (toy demo; real protocols hide Evan’s choice and the unchosen label). Evan inputs his bit `e`; Ginny holds both Evan labels and uses OT to deliver only the chosen one.


In [16]:
fn toy_ot(sender_l0: &[u8; 16], sender_l1: &[u8; 16], choice: u8) -> [u8; 16] {
    // Toy/semi-honest only: just returns the chosen label; does not hide choice or the other label.
    if choice == 0 {
        *sender_l0
    } else {
        *sender_l1
    }
}

// Evan picks his bit and obtains the matching label via the toy OT.
let evan_pref: u8 = if rand::random::<bool>() { 1 } else { 0 };
let evan_label = toy_ot(&garbled.input_b.zero, &garbled.input_b.one, evan_pref);
println!("Evan preference e={evan_pref}; OT delivers his input label:");
println!("  label (base64): `{}`", B64.encode(evan_label));


Evan preference e=0; OT delivers his input label:
  label (base64): `bL1zclBA2RXXL3IXR4uSaw`


With both input labels in hand (Ginny’s chosen label and Evan’s OT-delivered label), Evan decrypts the garbled table once to recover the AND result.


In [17]:
// Evan evaluates once using his label and Ginny's label; keep the output label/bit for sending back.
let mut evan_output_label: Option<[u8; 16]> = None;
let mut evan_output_bit: Option<u8> = None;
{
    let key = h(&ginny_label, &evan_label);
    for ct in &garbled.table {
        let out = encrypt(&key, ct);
        if out == garbled.out_labels.zero {
            evan_output_label = Some(out);
            evan_output_bit = Some(0);
            break;
        }
        if out == garbled.out_labels.one {
            evan_output_label = Some(out);
            evan_output_bit = Some(1);
            break;
        }
    }
}
let evan_output_label = evan_output_label.expect("no matching row decrypted");
let evan_output_bit = evan_output_bit.expect("no matching bit decrypted");
println!("Recovered AND result: g={} ∧ e={} -> {}", ginny_pref, evan_pref, evan_output_bit);


Recovered AND result: g=1 ∧ e=0 -> 0


Evan sends the decrypted output label back to Ginny.


In [18]:
// Evan sends the output label to Ginny (no re-computation).
println!("Evan sends output label to Ginny (base64): `{}`", B64.encode(evan_output_label));


Evan sends output label to Ginny (base64): `KlypN9WFJSS2UpgNKrxyZA`


Ginny maps the returned label to a bit using her output-label map.


In [19]:
// Ginny decodes the label she received.
{
    let ginny_decoded = if evan_output_label == garbled.out_labels.zero { 0 } else { 1 };
    println!("Ginny maps the label to bit: {}", ginny_decoded);
    assert_eq!(ginny_decoded, evan_output_bit);
}


Ginny maps the label to bit: 0


()